#🏥Hospital Admmissions Data EDA

Integrantes:
*   Martina Gonzalez
*   Raquel Rehbein




Preguntas de investigación
🔹 Clasificación

¿Podemos predecir si un paciente hospitalizado por una enfermedad cardiovascular fallecerá o será dado de alta, en función de sus características clínicas, demográficas y comorbilidades?

📘 Objetivo del modelo: predecir el desenlace hospitalario (OUTCOME: alta = 0 / fallecido = 1).

🔹 Regresión

¿Podemos estimar cuántos días permanecerá hospitalizado un paciente, considerando sus condiciones clínicas, demográficas y de ingreso?

📘 Objetivo del modelo: predecir la duración de la estancia hospitalaria (DURATION OF STAY, variable continua).

https://www.kaggle.com/datasets/ashishsahani/hospital-admissions-data

###Importar librerias y cargar datos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [ ]:
from google.colab import files
import pandas as pd

# Subir primer archivo admission
uploaded = files.upload()
df_adm = pd.read_csv(next(iter(uploaded)), sep=",", encoding="latin-1")

# Subir segundo archivo mortality
uploaded = files.upload()
df_mor = pd.read_csv(next(iter(uploaded)), sep=",", encoding="latin-1")

# Subir tercer archivo pollution
uploaded = files.upload()
df_pol = pd.read_csv(next(iter(uploaded)), sep=",", encoding="latin-1")


Saving HDHI Admission data.csv to HDHI Admission data (2).csv


Saving HDHI Mortality Data.csv to HDHI Mortality Data (2).csv


Saving HDHI Pollution Data.csv to HDHI Pollution Data (2).csv


###Revisión general

# 3️⃣Preparación de los datos

###Limpieza

In [ ]:
#limpieza de caracteres extraños, munisculas, uso de guion bajo en vez de espacios
def clean_columns(df):
    df.columns = df.columns.str.replace('ï»¿','').str.strip().str.lower().str.replace(" ", "_")
    return df

df_adm = clean_columns(df_adm)
df_mor = clean_columns(df_mor)
df_pol = clean_columns(df_pol)


In [ ]:
# Convertir columnas de fecha
df_adm['d.o.a'] = pd.to_datetime(df_adm['d.o.a'], errors='coerce')
df_adm['d.o.d'] = pd.to_datetime(df_adm['d.o.d'], errors='coerce')
df_mor['date_of_brought_dead'] = pd.to_datetime(df_mor['date_of_brought_dead'], errors='coerce')
df_pol['date'] = pd.to_datetime(df_pol['date'], errors='coerce')

# renombrar columnas para mayor comprension
df_adm.rename(columns={'d.o.a':'date_of_admission', 'd.o.d':'date_of_discharge'}, inplace=True)

In [ ]:
#conversion de columnas numericas que aparecian como tipo object
for col in ['hb','tlc','platelets','glucose','urea','creatinine','bnp','ef']:
    if col in df_adm.columns:
        df_adm[col] = pd.to_numeric(df_adm[col], errors='coerce')

for col in ['pm10_max','no2_max','ozone_max']:
    if col in df_pol.columns:
        df_pol[col] = pd.to_numeric(df_pol[col], errors='coerce')


Antes de tratar valores nulos, analizamos las columnas

In [ ]:
# Columnas con nulos en df_adm
adm_null_cols = ['hb','tlc','platelets','glucose','urea','creatinine','bnp','ef']
print("===== Estadísticas columnas df_adm con nulos =====")
print(df_adm[adm_null_cols].describe())

# Columnas con nulos en df_pol
pol_null_cols = ['humidity']
print("\n===== Estadísticas columnas df_pol con nulos =====")
print(df_pol[pol_null_cols].describe())


===== Estadísticas columnas df_adm con nulos =====
                 hb           tlc     platelets       glucose          urea  \
count  15501.000000  15467.000000  15463.000000  14812.000000  15513.000000   
mean      12.220635     11.524977    238.617025    162.902795     50.161188   
std        2.327900      7.472741    103.827157     84.371718     42.494672   
min        3.000000      0.100000      0.580000      1.200000      0.100000   
25%       10.700000      7.900000    171.000000    106.000000     25.000000   
50%       12.400000     10.100000    226.000000    136.000000     35.000000   
75%       13.900000     13.400000    288.000000    196.000000     58.000000   
max       26.500000    314.000000   1179.000000    888.000000    495.000000   

         creatinine          bnp            ef  
count  15506.000000  6676.000000  14158.000000  
mean       1.343636   817.847244     43.461252  
std        1.197744   997.756165     13.467014  
min        0.065000     4.000000     14.0

In [ ]:
# LIMPIEZA E IMPUTACIÓN DE NULOS
# --- Admission (df_adm) ---
# Columnas con pocos nulos → imputar con mediana
for col in ['hb','tlc','platelets','glucose','urea','creatinine','ef']:
    if col in df_adm.columns:
        median_value = df_adm[col].median()
        df_adm[col].fillna(median_value, inplace=True)
        print(f"Columna {col}: valores nulos imputados con mediana = {median_value}")

# Columnas con muchos nulos → eliminar del análisis inicial
if 'bnp' in df_adm.columns:
    df_adm.drop(columns=['bnp'], inplace=True)
    print("Columna BNP eliminada por tener demasiados valores nulos.")

# --- Pollution (df_pol) ---
# HUMIDITY: pocos nulos → imputar con media
if 'humidity' in df_pol.columns:
    mean_value = df_pol['humidity'].mean()
    df_pol['humidity'].fillna(mean_value, inplace=True)
    print(f"Columna HUMIDITY: valores nulos imputados con media = {mean_value:.2f}")

# --- Mortality (df_mor) ---
# No tiene nulos, nada que hacer
print("Mortality dataset no tiene valores nulos.")

# --- Confirmar limpieza ---
print("\nValores nulos restantes en df_adm:\n", df_adm.isnull().sum())
print("\nValores nulos restantes en df_pol:\n", df_pol.isnull().sum())
print("\nValores nulos restantes en df_mor:\n", df_mor.isnull().sum())


Columna hb: valores nulos imputados con mediana = 12.4
Columna tlc: valores nulos imputados con mediana = 10.1
Columna platelets: valores nulos imputados con mediana = 226.0
Columna glucose: valores nulos imputados con mediana = 136.0
Columna urea: valores nulos imputados con mediana = 35.0
Columna creatinine: valores nulos imputados con mediana = 1.0
Columna ef: valores nulos imputados con mediana = 42.0
Columna BNP eliminada por tener demasiados valores nulos.
Columna HUMIDITY: valores nulos imputados con media = 40.62
Mortality dataset no tiene valores nulos.

Valores nulos restantes en df_adm:
 sno                                   0
mrd_no.                               0
date_of_admission                  5655
date_of_discharge                  5550
age                                   0
gender                                0
rural                                 0
type_of_admission-emergency/opd       0
month_year                            0
duration_of_stay                  

/tmp/ipython-input-185793229.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_adm[col].fillna(median_value, inplace=True)
/tmp/ipython-input-185793229.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [ ]:
# Normalizar valores de rural/urban porque habian valores q se repetian
df_mor['rural/urban_clean'] = df_mor['rural/urban'].str.upper()  # todo en mayúscula
df_mor['rural/urban_clean'] = df_mor['rural/urban_clean'].replace({'0': None, 'M': None})  # convertir valores incorrectos a NaN


In [ ]:
# Merge df_adm and df_mor on a common key. Assuming 'mrd_no.' and 'mrd' are the common patient identifiers.
# We'll use a left merge from df_adm to keep all admission records.
df_adm_mor = pd.merge(df_adm, df_mor, left_on='mrd_no.', right_on='mrd', how='left')

# Display the first few rows and info to verify the merge
print("Shape after merging df_adm and df_mor:", df_adm_mor.shape)
display(df_adm_mor.head())
df_adm_mor.info()

# 2️⃣ Diabetes Mellitus (DM)
# The merge creates 'dm_x' and 'dm_y' if 'dm' exists in both dataframes.
# Use 'dm' from df_adm as it contains the admission information.
df_adm_mor['DIABETES'] = df_adm_mor['dm'].fillna(0).apply(lambda x: 1 if x > 0 else 0)

Shape after merging df_adm and df_mor: (15758, 62)


,sno,mrd_no.,date_of_admission,date_of_discharge,age_x,gender_x,rural,type_of_admission-emergency/opd,month_year,duration_of_stay,...,shock,pulmonary_embolism,chest_infection,s.no,mrd,age_y,gender_y,rural/urban,date_of_brought_dead,rural/urban_clean
0,1,234735,2017-04-01,2017-04-03,81,M,R,E,Apr-17,3,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,2,234696,2017-04-01,2017-04-05,65,M,R,E,Apr-17,5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,3,234882,2017-04-01,2017-04-03,53,M,U,E,Apr-17,3,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,4,234635,2017-04-01,2017-04-08,67,F,U,E,Apr-17,8,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,5,234486,2017-04-01,2017-04-23,60,F,U,E,Apr-17,23,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15758 entries, 0 to 15757
Data columns (total 62 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   sno                              15758 non-null  int64         
 1   mrd_no.                          15758 non-null  object        
 2   date_of_admission                10102 non-null  datetime64[ns]
 3   date_of_discharge                10207 non-null  datetime64[ns]
 4   age_x                            15758 non-null  int64         
 5   gender_x                         15758 non-null  object        
 6   rural                            15758 non-null  object        
 7   type_of_admission-emergency/opd  15758 non-null  object        
 8   month_year                       15758 non-null  object        
 9   duration_of_stay                 15758 non-null  int64         
 10  duration_of_intensive_unit_stay  15758 non-null  int64    

In [ ]:
# 3️⃣ Hipertensión (HTN)
df_adm_mor['HIPERTENSION'] = df_adm_mor['htn'].fillna(0).apply(lambda x: 1 if x>0 else 0)

In [ ]:
# 4️⃣ Insuficiencia renal (CKD)
df_adm_mor['INSUF_RENAL'] = df_adm_mor['ckd'].fillna(0).apply(lambda x: 1 if x>0 else 0)

In [ ]:
# 5️⃣ Enfermedad cerebrovascular (CVA)
cva_cols = ['cva_infract', 'cva_bleed']
df_adm_mor['ENF_CEREBROVASCULAR'] = df_adm_mor[cva_cols].fillna(0).sum(axis=1)
df_adm_mor['ENF_CEREBROVASCULAR'] = df_adm_mor['ENF_CEREBROVASCULAR'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
# 6️⃣ Insuficiencia cardíaca
df_adm_mor['INSUF_CARDIACA'] = df_adm_mor['heart_failure'].fillna(0).apply(lambda x: 1 if x>0 else 0)

In [ ]:
df_adm_mor.head(10)

,sno,mrd_no.,date_of_admission,date_of_discharge,age_x,gender_x,rural,type_of_admission-emergency/opd,month_year,duration_of_stay,...,age_y,gender_y,rural/urban,date_of_brought_dead,rural/urban_clean,DIABETES,HIPERTENSION,INSUF_RENAL,ENF_CEREBROVASCULAR,INSUF_CARDIACA
0,1,234735,2017-04-01,2017-04-03,81,M,R,E,Apr-17,3,...,NaN,NaN,NaN,NaT,NaN,1,0,0,0,1
1,2,234696,2017-04-01,2017-04-05,65,M,R,E,Apr-17,5,...,NaN,NaN,NaN,NaT,NaN,0,1,0,0,0
2,3,234882,2017-04-01,2017-04-03,53,M,U,E,Apr-17,3,...,NaN,NaN,NaN,NaT,NaN,1,0,0,0,1
3,4,234635,2017-04-01,2017-04-08,67,F,U,E,Apr-17,8,...,NaN,NaN,NaN,NaT,NaN,0,1,0,0,0
4,5,234486,2017-04-01,2017-04-23,60,F,U,E,Apr-17,23,...,NaN,NaN,NaN,NaT,NaN,0,1,0,0,0
5,6,234675,2017-04-01,2017-04-10,44,M,U,E,Apr-17,10,...,NaN,NaN,NaN,NaT,NaN,1,1,0,0,1
6,7,234563,2017-04-01,2017-04-06,56,F,U,E,Apr-17,6,...,NaN,NaN,NaN,NaT,NaN,1,1,0,0,1
7,8,208455,2017-04-01,2017-04-13,47,M,U,E,Apr-17,13,...,NaN,NaN,NaN,NaT,NaN,1,1,0,0,0
8,9,67070,2017-04-01,2017-04-03,65,F,U,E,Apr-17,3,...,NaN,NaN,NaN,NaT,NaN,0,1,0,0,1
9,10,153218,2017-04-01,2017-04-03,59,M,U,E,Apr-17,3,...,NaN,NaN,NaN,NaT,NaN,1,1,0,0,0


###Eliminacion de outliers

In [ ]:
def detectar_outliers(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[columna] < lower) | (df[columna] > upper)]
    print(f"Columna: {columna}")
    print(f"Rango aceptado: [{lower:.2f}, {upper:.2f}]")
    print(f"Cantidad de outliers: {len(outliers)}")
    return outliers

# Ejemplo con edad
outliers_age = detectar_outliers(df_adm_mor, "age_x")

# Ejemplo con duración de estancia
outliers_stay = detectar_outliers(df_adm_mor, "duration_of_stay")

Columna: age_x
Rango aceptado: [30.00, 94.00]
Cantidad de outliers: 362
Columna: duration_of_stay
Rango aceptado: [-4.50, 15.50]
Cantidad de outliers: 736


In [ ]:
for col in df_adm_mor.select_dtypes(include=["float64", "int64"]).columns:
    detectar_outliers(df_adm_mor, col)
    print("-"*40)

Columna: sno
Rango aceptado: [-7876.00, 23634.00]
Cantidad de outliers: 0
----------------------------------------
Columna: age_x
Rango aceptado: [30.00, 94.00]
Cantidad de outliers: 362
----------------------------------------
Columna: duration_of_stay
Rango aceptado: [-4.50, 15.50]
Cantidad de outliers: 736
----------------------------------------
Columna: duration_of_intensive_unit_stay
Rango aceptado: [-5.00, 11.00]
Cantidad de outliers: 683
----------------------------------------
Columna: smoking
Rango aceptado: [0.00, 0.00]
Cantidad de outliers: 793
----------------------------------------
Columna: alcohol
Rango aceptado: [0.00, 0.00]
Cantidad de outliers: 1021
----------------------------------------
Columna: dm
Rango aceptado: [-1.50, 2.50]
Cantidad de outliers: 0
----------------------------------------
Columna: htn
Rango aceptado: [-1.50, 2.50]
Cantidad de outliers: 0
----------------------------------------
Columna: cad
Rango aceptado: [-1.50, 2.50]
Cantidad de outliers: 0


Se eliminan outliers de edad menor a 20 y mayor a 100
Se eliminan outliers de duracion de estadia negativos y valores mayor a 100 dias
Se eliminan outliers de duracion de estadia en uci, negativos y mayores a 60 dias

In [ ]:
# -----------------------------
# 🔹 Eliminación de outliers
# -----------------------------

# 1. Edad: eliminar < 20 o > 100
df_adm_mor = df_adm_mor[(df_adm_mor['age_x'] >= 20) & (df_adm_mor['age_x'] <= 100)]

# 2. Duración de estancia: eliminar negativos o > 100
df_adm_mor = df_adm_mor[(df_adm_mor['duration_of_stay'] >= 0) & (df_adm_mor['duration_of_stay'] <= 100)]

# 3. Duración en UCI: eliminar negativos o > 60
df_adm_mor = df_adm_mor[(df_adm_mor['duration_of_intensive_unit_stay'] >= 0) & (df_adm_mor['duration_of_intensive_unit_stay'] <= 60)]

# Resetear índice tras limpieza
df_adm_mor = df_adm_mor.reset_index(drop=True)

print("✅ Shape final después de limpiar:", df_adm_mor.shape)

✅ Shape final después de limpiar: (15672, 67)


##**MODELO1**

Evaluar un modelo de **RandomForestRegressor** como punto de partida para predecir la **duración de la estancia hospitalaria (`DURATION_OF_STAY`)**, analizando su desempeño y limitaciones.


aaa


In [ ]:
import pandas as pd

# --- Cargar el dataset de contaminación correctamente ---
df_pollution = pd.read_csv("/content/HDHI Pollution Data.csv", encoding="utf-8-sig")

# Verificamos la columna
print(df_pollution.columns[0])  # debería imprimir 'DATE' ahora

# --- Crear columna month_year (mes-año) ---
df_pollution['DATE'] = pd.to_datetime(df_pollution['DATE'], errors='coerce')
df_pollution['month_year'] = df_pollution['DATE'].dt.to_period('M').astype(str)

# 1️⃣ Convertir month_year de df_adm_mor a formato 'YYYY-MM'
df_adm_mor['month_year'] = pd.to_datetime(df_adm_mor['month_year'], format='%b-%y').dt.strftime('%Y-%m')


# 4️⃣ Hacer el merge usando month_year como clave
df_final = df_adm_mor.merge(df_pollution, on='month_year', how='left')

# 5️⃣ Verificar resultado
print(df_final.shape)
print(df_final.head())

DATE
(476652, 94)
   sno mrd_no. date_of_admission date_of_discharge  age_x gender_x rural  \
0    1  234735        2017-04-01        2017-04-03     81        M     R   
1    1  234735        2017-04-01        2017-04-03     81        M     R   
2    1  234735        2017-04-01        2017-04-03     81        M     R   
3    1  234735        2017-04-01        2017-04-03     81        M     R   
4    1  234735        2017-04-01        2017-04-03     81        M     R   

  type_of_admission-emergency/opd month_year  duration_of_stay  ...  CO AVG  \
0                               E    2017-04                 3  ...      40   
1                               E    2017-04                 3  ...      30   
2                               E    2017-04                 3  ...      28   
3                               E    2017-04                 3  ...      32   
4                               E    2017-04                 3  ...      29   

  CO MIN  CO MAX  OZONE AVG  OZONE MIN  OZONE MAX 

In [ ]:
import pandas as pd

# --- Cargar el dataset de contaminación correctamente ---
df_pollution = pd.read_csv("/content/HDHI Pollution Data.csv", encoding="utf-8-sig")

# Verificamos la columna
print(df_pollution.columns[0])

# --- Crear columna month_year (mes-año) ---
df_pollution['DATE'] = pd.to_datetime(df_pollution['DATE'], errors='coerce')
df_pollution['month_year'] = df_pollution['DATE'].dt.to_period('M').astype(str)

# --- Verificamos coincidencia con df_adm_mor ---
# The month_year in df_adm_mor is already in 'YYYY-MM' format from a previous step.
# No need to convert it again.
print(df_adm_mor['month_year'].unique()[:5])
print(df_pollution['month_year'].unique()[:5])

# --- Merge (unión) ---
df_final = pd.merge(
    df_adm_mor,
    df_pollution[['month_year', 'PM2.5 AVG', 'PM10 AVG', 'NO2 AVG', 'SO2 AVG', 'AQI']],
    on='month_year',
    how='left'
)

# --- Resultado ---
print("✅ Dataset combinado:", df_final.shape)
df_final.head()

DATE
['2017-04' '2017-05' '2017-06' '2017-07' '2017-08']
['2017-03' '2017-04' '2017-05' '2017-06' '2017-07']
✅ Dataset combinado: (476652, 72)


,sno,mrd_no.,date_of_admission,date_of_discharge,age_x,gender_x,rural,type_of_admission-emergency/opd,month_year,duration_of_stay,...,DIABETES,HIPERTENSION,INSUF_RENAL,ENF_CEREBROVASCULAR,INSUF_CARDIACA,PM2.5 AVG,PM10 AVG,NO2 AVG,SO2 AVG,AQI
0,1,234735,2017-04-01,2017-04-03,81,M,R,E,2017-04,3,...,1,0,0,0,1,38,35,20,4,52
1,1,234735,2017-04-01,2017-04-03,81,M,R,E,2017-04,3,...,1,0,0,0,1,53,59,18,6,59
2,1,234735,2017-04-01,2017-04-03,81,M,R,E,2017-04,3,...,1,0,0,0,1,62,70,10,9,70
3,1,234735,2017-04-01,2017-04-03,81,M,R,E,2017-04,3,...,1,0,0,0,1,70,63,12,9,70
4,1,234735,2017-04-01,2017-04-03,81,M,R,E,2017-04,3,...,1,0,0,0,1,88,85,6,6,88


In [ ]:
print("📅 df_adm_mor month_year únicos:")
print(df_adm_mor['month_year'].unique()[:10])

print("\n🌫️ df_pollution month_year únicos:")
print(df_pollution['month_year'].unique()[:10])


📅 df_adm_mor month_year únicos:
['2017-04' '2017-05' '2017-06' '2017-07' '2017-08' '2017-09' '2017-10'
 '2017-11' '2017-12' '2018-01']

🌫️ df_pollution month_year únicos:
['2017-03' '2017-04' '2017-05' '2017-06' '2017-07' '2017-08' '2017-09'
 '2017-10' '2017-11' '2017-12']


In [ ]:
# Copia de seguridad del dataset
df_copy = df_final.copy()

In [ ]:
import pandas as pd
import numpy as np

# Asegurar que duration_of_intensive_unit_stay sea numérico
df_final['duration_of_intensive_unit_stay'] = pd.to_numeric(df_final['duration_of_intensive_unit_stay'], errors='coerce')

# Seleccionar solo columnas numéricas
numeric_df = df_final.select_dtypes(include=[np.number])

# Calcular matriz de correlaciones
corr_matrix = numeric_df.corr()

# Obtener correlaciones con duration_of_intensive_unit_stay (y ordenar de mayor a menor)
corr_target = corr_matrix['duration_of_intensive_unit_stay'].drop('duration_of_intensive_unit_stay').sort_values(ascending=False)

# Mostrar el top 40
top40 = corr_target.head(50)
print(top40)




duration_of_stay             0.795924
heart_failure                0.206422
INSUF_CARDIACA               0.206422
urea                         0.205073
aki                          0.189609
anaemia                      0.168973
creatinine                   0.162145
tlc                          0.153090
raised_cardiac_enzymes       0.144576
hfref                        0.142331
glucose                      0.129381
INSUF_RENAL                  0.127429
ckd                          0.127429
hfnef                        0.123729
age_x                        0.121774
acs                          0.110855
stemi                        0.110409
vt                           0.107690
prior_cmp                    0.095770
chb                          0.092372
af                           0.069041
uti                          0.064604
shock                        0.062765
cardiogenic_shock            0.055620
dm                           0.050270
DIABETES                     0.050270
ENF_CEREBROV

In [ ]:
# Variables
target = 'duration_of_intensive_unit_stay'

# Features con correlación positiva (depuradas y sin duplicadas)
features = [
    'duration_of_stay',              # días totales de hospitalización
    'heart_failure',                 # insuficiencia cardíaca
    'aki',                           # lesión renal aguda
    'anaemia',                       # anemia
    'raised_cardiac_enzymes',        # enzimas cardíacas elevadas
    'hfref',                         # fracción de eyección reducida
    'ckd',                           # enfermedad renal crónica
    'hfnef',                         # fracción de eyección preservada
    'age_x',                           # edad
    'acs',                           # síndrome coronario agudo
    'stemi',                         # infarto agudo de miocardio
    'vt',                            # taquicardia ventricular
    'prior_cmp',                     # cardiopatía previa
    'chb',                           # bloqueo auriculoventricular
    'af',                            # fibrilación auricular
    'uti',                           # infección urinaria
    'shock',                         # shock no cardiogénico
    'cardiogenic_shock',             # shock cardiogénico
    'dm',                            # diabetes mellitus
    'ENF_CEREBROVASCULAR',           # enfermedad cerebrovascular
    'cva_infract',                   # infarto cerebral
    'pulmonary_embolism',            # embolia pulmonar
    'orthostatic',                   # hipotensión ortostática
    'severe_anaemia',                # anemia severa
    'sno',                           # síncope no ortostático
    'cad',                           # enfermedad coronaria
    'cva_bleed',                     # hemorragia cerebral
    'sss',                           # síndrome de Stokes-Adams
    'dvt',                           # trombosis venosa profunda
    'infective_endocarditis',        # endocarditis infecciosa
    'SO2 AVG',                       # promedio saturación de oxígeno
    'valvular'                       # enfermedad valvular
]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
X = df_copy[features]
y = df_copy[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
rf_model = RandomForestRegressor(
    n_estimators=100,      # número de árboles
    max_depth=None,        # profundidad ilimitada por ahora
    random_state=42
)
rf_model.fit(X_train, y_train)

# Predicciones
y_pred = rf_model.predict(X_test)

# Evaluar
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("🌲 Random Forest Regression — Duración de Estancia en UCI")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")


🌲 Random Forest Regression — Duración de Estancia en UCI
MSE: 0.0001
RMSE: 0.0074
R²: 1.0000


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf_model, X, y, scoring='r2', cv=5)
print("R² promedio en validación cruzada:", np.mean(scores))


R² promedio en validación cruzada: 0.5149543002576216


📊 Interpretación

R² promedio en validación cruzada = 0.515

Significa que, cuando el modelo se evalúa en datos nuevos (no vistos), solo logra explicar el 51,5% de la variabilidad de la duración en UCI.

💬 En otras palabras:

El modelo parecía perfecto (R² = 1.0) porque memorizó el conjunto de entrenamiento,
pero en realidad su rendimiento real es moderado, no perfecto.

🛠 Solución: ajustar el modelo para mejorar generalización

In [ ]:
rf_model_tuned = RandomForestRegressor(
    n_estimators=300,
    max_depth=7,
    min_samples_leaf=5,
    min_samples_split=5,
    random_state=42
)
rf_model_tuned.fit(X_train, y_train)

# Evaluación
y_pred_tuned = rf_model_tuned.predict(X_test)

mse = mean_squared_error(y_test, y_pred_tuned)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_tuned)

print("🌲 Random Forest Ajustado — Duración de Estancia en UCI")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")


🌲 Random Forest Ajustado — Duración de Estancia en UCI
MSE: 4.6378
RMSE: 2.1536
R²: 0.7150


In [ ]:
scores = cross_val_score(rf_model_tuned, X, y, scoring='r2', cv=5)
print("R² promedio en validación cruzada:", np.mean(scores))


R² promedio en validación cruzada: 0.6116222937232088
